In [1]:
import gzip
import json
import time
import requests 
import pandas as pd
import logging as log
from configparser import RawConfigParser

class Collector():
    
    URL = 'https://neuvoo.com/services/api-new/search?' + \
        'ip=1.1.1.1&useragent=123asd&' + \
        'k=${JOB_TITLE}& ' + \
        'l=${CITY}&f=&o=&p=&r=15&' + \
        'country=${COUNTRY}' + \
        '&format=json&jobdesc=1' + \
        '&start=${START}' + \
        '&limit=${LIMIT}'

    #limit jobs post per resquest (defalt and max = 15)
    LIMIT = 15
    
    #max jobs post per occupation
    MAX = 1000
       
    def __init__(self, args):
        #credentials file
        config = RawConfigParser()
        config.read('./credentials.properties')
     
        self.args = args
        self.username = config.get('ApiSection', 'neuvoo.username')
        self.password = config.get('ApiSection', 'neuvoo.password')
  
        #print(self.username, self.password, self.args['input_file'], self.args['output_file'], self.URL)
    
    def get_job_request(self, job_title, start, limit, location):
        #country, city = location.split(',')
        return requests.get(self.URL
                            .replace("${JOB_TITLE}", job_title)
                            .replace("${START}", start)
                            .replace("${LIMIT}", limit)
                            .replace("${COUNTRY}", 'FR')
                            .replace("${CITY}", location),
                            auth=(self.username, self.password))
    

    def get_jobs(self, job_title, location):
        print("Getting jobs for: " + job_title + " - "+ location)
        data = pd.DataFrame()
        job_titles = [] 
        job_titles_found = []
        companies = []
        descriptions = []
        locations = []
        sources = [] 
        jobkeys = []
        dates = []
        department_name = []
        department_code = []

        for i in range(0, self.MAX, self.LIMIT):
            results = self.get_job_request(job_title, str(i), str(self.LIMIT), location)
            #results_count = len(results.json()['results'])
            totalresults = results.json()['totalresults']
            if totalresults != None:
                for result in results.json()['results']:
                    job_titles.append(job_title)
                    job_titles_found.append(result['jobtitle'])
                    companies.append(result['company'])
                    descriptions.append(result['description'])
                    locations.append(result['formattedLocation'])
                    sources.append(result['source'])
                    jobkeys.append(result['jobkey'])
                    dates.append(result['date'])
                    department_name.append(location)
                    department_code.append(get_department_code(location))
            else:
                break

        data["job_title"] = job_titles
        data["job_title_found"] = job_titles_found
        data["company"] = companies
        data["location"] = locations
        data["sources"] = sources
        data["jobkeys"] = jobkeys
        data["dates"] = dates
        data["description"] = descriptions
        data["department_name"] = department_name
        data["department_code"] = department_code

        return data
    
    def run(self):
        ts = time.gmtime()
        df = pd.DataFrame()
        for location in self.args['locations']:
            df = df.append([self.get_jobs(job_title, location) for job_title in self.args['jobs']])
        
        #tmp_output_file = args['output_file'].replace("${TS}", time.strftime("%Y%m%d%H%M%S", ts))
        #df.to_csv(tmp_output_file, index=False, compression='gzip')

        #print("File saved: ", tmp_output_file)

        return df

In [2]:
departments = pd.read_excel('data/department code.xlsx')
departments = departments[1:]
departments

,NUMÉRO,NOM
1,1,Ain
2,2,Aisne
3,3,Allier
4,4,Alpes-de-Haute-Provence
5,5,Hautes-Alpes
6,6,Alpes-Maritimes
7,7,Ardèche
8,8,Ardennes
9,9,Ariège
10,10,Aube


In [3]:
def get_department_code(name):
    return departments[departments['NOM'].str.startswith(name)]['NUMÉRO'].values[0]
    
get_department_code('Somme')

80

In [11]:
rome_codes = pd.read_csv('data/Sncf codes romes - Sheet1.csv', names=['title', 'rome_code'])
rome_codes

,title,rome_code
0,Agent technique d'unité patrimoine,C1501
1,Assistant technique Patrimoine,C1501
2,Gestionnaire technique immobilier,C1501
3,Assistant de production,C1502
4,Gestionnaire du fichier immobilier,C1502
5,Gestionnaire locatif,C1502
6,Pilote d'opération,C1503
7,Développeur d'opérations immobilières,C1503
8,Télé conseiller (H/F),D1408
9,Communicant,E1103


In [5]:
def get_rome_code(title):
    return rome_codes[rome_codes['title'].str.startswith(title)]['rome_code'].values[0]
    
get_rome_code('Assistant de production')

'C1502'

In [12]:
JOB_TITLES = rome_codes['title'][110:]
#JOB_TITLES = ['Assistant de production']

LOCATIONS = departments['NOM'].values
#LOCATIONS = ['Paris']

#arguments
args = {
    'name':'neuvoo',
    'jobs': JOB_TITLES,
    'locations': LOCATIONS,
    'output_file': '../../../data/raw/neuvoo/${TS}.csv.gz',
    #...
}

collector = Collector(args)
df = collector.run()
df

Getting jobs for: Dirigeant de Proximité télécoms - Ain
Getting jobs for: Responsable d'applications - Ain
Getting jobs for: Assistant au maitre d'ouvrage - Ain
Getting jobs for: Responsable MOA - Ain
Getting jobs for: Chef de projet Voyageurs - Ain
Getting jobs for: Directeur d'établissement Voyageurs - Ain
Getting jobs for: Dirigeant d'activité voyageurs - Ain
Getting jobs for: Dirigeant d'unité opérationnelle Voyageurs - Ain
Getting jobs for: ER non répertorié Voyageurs - Ain
Getting jobs for: Chargé de tarification distribution billétique - Ain
Getting jobs for: Agent d'escale et de service en gare - Ain
Getting jobs for: Agent de service et d'Information Voyageurs - Ain
Getting jobs for: Chef d'escale - Ain
Getting jobs for: Dirigeant de proximité produit train en gare - Ain
Getting jobs for: DPX ventes, services en gare, à la clientèle - Ain
Getting jobs for: ER non répertorié Transverses - Ain
Getting jobs for: Dirigeant de Proximité télécoms - Aisne
Getting jobs for: Responsabl

Getting jobs for: Responsable d'applications - Ariège
Getting jobs for: Assistant au maitre d'ouvrage - Ariège
Getting jobs for: Responsable MOA - Ariège
Getting jobs for: Chef de projet Voyageurs - Ariège
Getting jobs for: Directeur d'établissement Voyageurs - Ariège
Getting jobs for: Dirigeant d'activité voyageurs - Ariège
Getting jobs for: Dirigeant d'unité opérationnelle Voyageurs - Ariège
Getting jobs for: ER non répertorié Voyageurs - Ariège
Getting jobs for: Chargé de tarification distribution billétique - Ariège
Getting jobs for: Agent d'escale et de service en gare - Ariège
Getting jobs for: Agent de service et d'Information Voyageurs - Ariège
Getting jobs for: Chef d'escale - Ariège
Getting jobs for: Dirigeant de proximité produit train en gare - Ariège
Getting jobs for: DPX ventes, services en gare, à la clientèle - Ariège
Getting jobs for: ER non répertorié Transverses - Ariège
Getting jobs for: Dirigeant de Proximité télécoms - Aube
Getting jobs for: Responsable d'applicat

Getting jobs for: Dirigeant d'activité voyageurs - Charente-Maritime
Getting jobs for: Dirigeant d'unité opérationnelle Voyageurs - Charente-Maritime
Getting jobs for: ER non répertorié Voyageurs - Charente-Maritime
Getting jobs for: Chargé de tarification distribution billétique - Charente-Maritime
Getting jobs for: Agent d'escale et de service en gare - Charente-Maritime
Getting jobs for: Agent de service et d'Information Voyageurs - Charente-Maritime
Getting jobs for: Chef d'escale - Charente-Maritime
Getting jobs for: Dirigeant de proximité produit train en gare - Charente-Maritime
Getting jobs for: DPX ventes, services en gare, à la clientèle - Charente-Maritime
Getting jobs for: ER non répertorié Transverses - Charente-Maritime
Getting jobs for: Dirigeant de Proximité télécoms - Cher
Getting jobs for: Responsable d'applications - Cher
Getting jobs for: Assistant au maitre d'ouvrage - Cher
Getting jobs for: Responsable MOA - Cher
Getting jobs for: Chef de projet Voyageurs - Cher
G

Getting jobs for: Dirigeant d'unité opérationnelle Voyageurs - Dordogne
Getting jobs for: ER non répertorié Voyageurs - Dordogne
Getting jobs for: Chargé de tarification distribution billétique - Dordogne
Getting jobs for: Agent d'escale et de service en gare - Dordogne
Getting jobs for: Agent de service et d'Information Voyageurs - Dordogne
Getting jobs for: Chef d'escale - Dordogne
Getting jobs for: Dirigeant de proximité produit train en gare - Dordogne
Getting jobs for: DPX ventes, services en gare, à la clientèle - Dordogne
Getting jobs for: ER non répertorié Transverses - Dordogne
Getting jobs for: Dirigeant de Proximité télécoms - Doubs
Getting jobs for: Responsable d'applications - Doubs
Getting jobs for: Assistant au maitre d'ouvrage - Doubs
Getting jobs for: Responsable MOA - Doubs
Getting jobs for: Chef de projet Voyageurs - Doubs
Getting jobs for: Directeur d'établissement Voyageurs - Doubs
Getting jobs for: Dirigeant d'activité voyageurs - Doubs
Getting jobs for: Dirigeant

Getting jobs for: Dirigeant de proximité produit train en gare - Gers
Getting jobs for: DPX ventes, services en gare, à la clientèle - Gers
Getting jobs for: ER non répertorié Transverses - Gers
Getting jobs for: Dirigeant de Proximité télécoms - Gironde
Getting jobs for: Responsable d'applications - Gironde
Getting jobs for: Assistant au maitre d'ouvrage - Gironde
Getting jobs for: Responsable MOA - Gironde
Getting jobs for: Chef de projet Voyageurs - Gironde
Getting jobs for: Directeur d'établissement Voyageurs - Gironde
Getting jobs for: Dirigeant d'activité voyageurs - Gironde
Getting jobs for: Dirigeant d'unité opérationnelle Voyageurs - Gironde
Getting jobs for: ER non répertorié Voyageurs - Gironde
Getting jobs for: Chargé de tarification distribution billétique - Gironde
Getting jobs for: Agent d'escale et de service en gare - Gironde
Getting jobs for: Agent de service et d'Information Voyageurs - Gironde
Getting jobs for: Chef d'escale - Gironde
Getting jobs for: Dirigeant de 

...

Getting jobs for: ER non répertorié Voyageurs - Savoie
Getting jobs for: Chargé de tarification distribution billétique - Savoie
Getting jobs for: Agent d'escale et de service en gare - Savoie
Getting jobs for: Agent de service et d'Information Voyageurs - Savoie
Getting jobs for: Chef d'escale - Savoie
Getting jobs for: Dirigeant de proximité produit train en gare - Savoie
Getting jobs for: DPX ventes, services en gare, à la clientèle - Savoie
Getting jobs for: ER non répertorié Transverses - Savoie
Getting jobs for: Dirigeant de Proximité télécoms - Haute-Savoie
Getting jobs for: Responsable d'applications - Haute-Savoie
Getting jobs for: Assistant au maitre d'ouvrage - Haute-Savoie
Getting jobs for: Responsable MOA - Haute-Savoie
Getting jobs for: Chef de projet Voyageurs - Haute-Savoie
Getting jobs for: Directeur d'établissement Voyageurs - Haute-Savoie
Getting jobs for: Dirigeant d'activité voyageurs - Haute-Savoie
Getting jobs for: Dirigeant d'unité opérationnelle Voyageurs - Hau

Getting jobs for: Agent d'escale et de service en gare - Tarn
Getting jobs for: Agent de service et d'Information Voyageurs - Tarn
Getting jobs for: Chef d'escale - Tarn
Getting jobs for: Dirigeant de proximité produit train en gare - Tarn
Getting jobs for: DPX ventes, services en gare, à la clientèle - Tarn
Getting jobs for: ER non répertorié Transverses - Tarn
Getting jobs for: Dirigeant de Proximité télécoms - Tarn-et-Garonne
Getting jobs for: Responsable d'applications - Tarn-et-Garonne
Getting jobs for: Assistant au maitre d'ouvrage - Tarn-et-Garonne
Getting jobs for: Responsable MOA - Tarn-et-Garonne
Getting jobs for: Chef de projet Voyageurs - Tarn-et-Garonne
Getting jobs for: Directeur d'établissement Voyageurs - Tarn-et-Garonne
Getting jobs for: Dirigeant d'activité voyageurs - Tarn-et-Garonne
Getting jobs for: Dirigeant d'unité opérationnelle Voyageurs - Tarn-et-Garonne
Getting jobs for: ER non répertorié Voyageurs - Tarn-et-Garonne
Getting jobs for: Chargé de tarification di

Getting jobs for: ER non répertorié Transverses - Yonne
Getting jobs for: Dirigeant de Proximité télécoms - Territoire de Belfort
Getting jobs for: Responsable d'applications - Territoire de Belfort
Getting jobs for: Assistant au maitre d'ouvrage - Territoire de Belfort
Getting jobs for: Responsable MOA - Territoire de Belfort
Getting jobs for: Chef de projet Voyageurs - Territoire de Belfort
Getting jobs for: Directeur d'établissement Voyageurs - Territoire de Belfort
Getting jobs for: Dirigeant d'activité voyageurs - Territoire de Belfort
Getting jobs for: Dirigeant d'unité opérationnelle Voyageurs - Territoire de Belfort
Getting jobs for: ER non répertorié Voyageurs - Territoire de Belfort
Getting jobs for: Chargé de tarification distribution billétique - Territoire de Belfort
Getting jobs for: Agent d'escale et de service en gare - Territoire de Belfort
Getting jobs for: Agent de service et d'Information Voyageurs - Territoire de Belfort
Getting jobs for: Chef d'escale - Territoire

Getting jobs for: Agent de service et d'Information Voyageurs - Martinique
Getting jobs for: Chef d'escale - Martinique
Getting jobs for: Dirigeant de proximité produit train en gare - Martinique
Getting jobs for: DPX ventes, services en gare, à la clientèle - Martinique
Getting jobs for: ER non répertorié Transverses - Martinique
Getting jobs for: Dirigeant de Proximité télécoms - Guyane
Getting jobs for: Responsable d'applications - Guyane
Getting jobs for: Assistant au maitre d'ouvrage - Guyane
Getting jobs for: Responsable MOA - Guyane
Getting jobs for: Chef de projet Voyageurs - Guyane
Getting jobs for: Directeur d'établissement Voyageurs - Guyane
Getting jobs for: Dirigeant d'activité voyageurs - Guyane
Getting jobs for: Dirigeant d'unité opérationnelle Voyageurs - Guyane
Getting jobs for: ER non répertorié Voyageurs - Guyane
Getting jobs for: Chargé de tarification distribution billétique - Guyane
Getting jobs for: Agent d'escale et de service en gare - Guyane
Getting jobs for: 

,job_title,job_title_found,company,location,sources,jobkeys,dates,description,department_name,department_code
0,Responsable d'applications,RESPONSABLE DEVELOPPEMENT APPLICATIONSH/F – AL...,FC Recrutement,"null, Grand Est, FR",FC Recrutement,76b65f664620,2020-07-15,Expérience requise 3 à 7 ans Diplôme Bac+5 Pré...,Ardennes,8.0
0,Assistant au maitre d'ouvrage,Assistant au Maître d'ouvrage,SNCF,"null, Provence-Alpes-Côte d'Azur, FR",SNCF,2f1f81b49410,2020-07-13,Vos missions MISSION PRINCIPALE :L'alternant a...,Bouches-du-Rhône,13.0
0,Responsable MOA,Responsable d'opérations MOA F/H,Hays France,"Marseille, Provence-Alpes-Côte d'Azur, FR",Apec,e9282237f15b,2020-07-10,"Rattaché à la direction technique, vous interv...",Bouches-du-Rhône,13.0
0,Responsable d'applications,Technicien intégrateur - Responsable d’applica...,Calvados,"Bénouville, Lower Normandy, FR",Calvados,478d2a8ea308,2020-07-16,Votre quotidien: Résolument inscrite dans la t...,Calvados,14.0
1,Responsable d'applications,Offre N° 23784 Technicien intég...,Département du Calvados,"Caen, Normandy, FR",Département du Calvados,05eb12389ccc,2020-07-16,Votre quotidien: Résolument inscrite dans la t...,Calvados,14.0
0,Responsable d'applications,Responsable Applications Métiers (F/H),Loc Maria Biscuits,"Dinan, Brittany, FR",Loc Maria Biscuits,987cadcd2f84,2020-07-13,"LOC MARIA BISCUITS, groupe de 410 salariés pr...",Côtes d'Armor,22.0
0,Responsable d'applications,Responsable Sécurité Web et Applications (H/F),Lapeyre,"null, null, FR",Lapeyre,c69e41cc632c,2020-07-16,Au sein du service IT Architecture et Gouverna...,Creuse,23.0
0,Responsable d'applications,Responsable d'applications et de projets IT H/F,Crédit Mutuel Arkéa,"Brest, Brittany, FR",Arkea,19d7cbe0d801,2020-07-10,Votre missionIntégrer Arkéa en tant que Respon...,Finistère,29.0
1,Responsable d'applications,Responsable d&apos;applications et de projets ...,Crédit Mutuel Arkéa,"Brest, Brittany, FR",Linkfinance,bbc547c9b01f,2020-07-14,Intégrer Arkéa en tant que Responsable d'appli...,Finistère,29.0
2,Responsable d'applications,Responsable d'applications et de projets it H/F,Linkfinance,"Brest, Brittany, FR",Linkfinance,f15f43107f7f,2020-07-10,Intégrer Arkéa en tant que Responsable d’appli...,Finistère,29.0


In [13]:
df['rome_code'] = [get_rome_code(title) for title in df['job_title']]
df.head()

,job_title,job_title_found,company,location,sources,jobkeys,dates,description,department_name,department_code,rome_code
0,Responsable d'applications,RESPONSABLE DEVELOPPEMENT APPLICATIONSH/F – AL...,FC Recrutement,"null, Grand Est, FR",FC Recrutement,76b65f664620,2020-07-15,Expérience requise 3 à 7 ans Diplôme Bac+5 Pré...,Ardennes,8.0,M1805
0,Assistant au maitre d'ouvrage,Assistant au Maître d'ouvrage,SNCF,"null, Provence-Alpes-Côte d'Azur, FR",SNCF,2f1f81b49410,2020-07-13,Vos missions MISSION PRINCIPALE :L'alternant a...,Bouches-du-Rhône,13.0,M1806
0,Responsable MOA,Responsable d'opérations MOA F/H,Hays France,"Marseille, Provence-Alpes-Côte d'Azur, FR",Apec,e9282237f15b,2020-07-10,"Rattaché à la direction technique, vous interv...",Bouches-du-Rhône,13.0,M1806
0,Responsable d'applications,Technicien intégrateur - Responsable d’applica...,Calvados,"Bénouville, Lower Normandy, FR",Calvados,478d2a8ea308,2020-07-16,Votre quotidien: Résolument inscrite dans la t...,Calvados,14.0,M1805
1,Responsable d'applications,Offre N° 23784 Technicien intég...,Département du Calvados,"Caen, Normandy, FR",Département du Calvados,05eb12389ccc,2020-07-16,Votre quotidien: Résolument inscrite dans la t...,Calvados,14.0,M1805


In [14]:
df.to_clipboard(index=False)

In [19]:
df.loc[df.job_title == 'promoteur des ventes', 'rome_code'] = 'M1706'
df.loc[df.job_title == 'responsable de secteur', 'rome_code'] = 'D1509'

df

,job_title,job_title_found,company,location,sources,jobkeys,dates,description,department_name,department_code,rome_code
0,Télé conseiller (H/F),Conseiller marketing relationnel et e-commerce...,Distributeur Indépendant,"Gex, Auvergne-Rhône-Alpes, FR",PMEjob.fr,4cf804add098,2020-07-14,La société :Nous travaillons en partenariat av...,Ain,1,NaN
0,Télé conseiller (H/F),CONSEILLER DE VENTE CDD - SAINT QUENTIN FAYET H/F,Bouygues Telecom,"Saint-Quentin, Hauts-de-France, FR",Direct Emploi,cfd3ab58c2e9,2020-07-10,#nosclientdabord est devenu un moteur de l&#39...,Aisne,2,NaN
1,Télé conseiller (H/F),Conseiller marketing relationnel et e-commerce...,Distributeur Indépendant,"Laon, Hauts-de-France, FR",Jobposting,b1cf082e6045,2020-07-09,Nous travaillons en partenariat avec une grand...,Aisne,2,NaN
2,Télé conseiller (H/F),Conseiller commercial beauté et e-boutique tél...,Distributeur Indépendant,"Laon, Hauts-de-France, FR",PMEjob.fr,fa237daa7c98,2020-07-11,La société :Nous travaillons en partenariat av...,Aisne,2,NaN
3,Télé conseiller (H/F),Conseillère beauté et télétravail e-boutique H/F,Distributeur Indépendant,"Laon, Hauts-de-France, FR",Jobposting,0464598b79cd,2020-07-12,Nous travaillons en partenariat avec une grand...,Aisne,2,NaN
0,Télé conseiller (H/F),Conseillère beauté e-commerce télétravail H/F,Distributeur Indépendant,"Vichy, Auvergne-Rhône-Alpes, FR",Jobposting,4bd75bd7b799,2020-07-15,Nous travaillons en partenariat avec une grand...,Allier,3,NaN
1,Télé conseiller (H/F),Conseiller commercial beauté et e-boutique tél...,Distributeur Indépendant,"Montluçon, Auvergne-Rhône-Alpes, FR",Jobposting,27c26b48ab02,2020-07-10,Nous travaillons en partenariat avec une grand...,Allier,3,NaN
2,Télé conseiller (H/F),Conseiller marketing relationnel et e-commerce...,Distributeur Indépendant,"Moulins, Auvergne-Rhône-Alpes, FR",Jobposting,d4eec3500987,2020-07-09,Nous travaillons en partenariat avec une grand...,Allier,3,NaN
3,Télé conseiller (H/F),CONSEILLER DE VENTE CDD ETE - MONTLUCON H/F,Bouygues Telecom,"Montluçon, Auvergne-Rhône-Alpes, FR",Bouygues Telecom,24e327c18bac,2020-07-10,Description de la mission Chez Bouygues Teleco...,Allier,3,NaN
0,Télé conseiller (H/F),Conseillère commerciale - Conseiller commercia...,Orange,"Arrondissement of Gap, Provence-Alpes-Côte d'A...",Orange,95d8962fe9b3,2020-07-09,about the role Nous vous proposons un contrat ...,Hautes-Alpes,5,NaN


In [44]:
#Status Code, Content Type, Enconding, Total Results
#r.status_code, r.headers['content-type'], r.encoding, r.json()['totalresults']

result = collector.get_job_request('Data scientist', '0', '15', "us,Silicon Valley")
assert result.status_code == 200
result.json()['results']

In [46]:
collector.get_job_request('ventes', '0', '15', "FR, Paris").json()

{'totalresults': 211,
 'results': [{'jobkey': 'ce45ab23fa3b',
   'jobtitle': 'Chef de ventes Véhicules Neufs H/F Région parisienne',
   'company': 'RENAULT RETAIL GROUP',
   'city': 'Paris',
   'state': 'Île-de-France',
   'country': 'FR',
   'formattedLocation': 'Paris, Île-de-France, FR',
   'source': 'RENAULT RETAIL GROUP',
   'date': '2020-07-02',
   'description': "Des métiers « passion », des produits « plaisir », des possibilités de s'épanouir professionnellement dans un environnement varié et dynamique centré sur nos clients et leur satisfaction…Renault Retail Group est une filiale à 100% du constructeur Renault. Le monde de l'automobile est en perpétuelle évolution et nous nous attachons à la conception de véhicules innovants, séduisants et encore plus respectueux de l'environnement. RRG repose sur nos valeurs telles que : garder toujours le client à l'esprit, apporter sa contribution, parler vrai, apprendre au quotidien et faire simple. Notre culture d'entreprise est notre fo

In [47]:
collector.get_job_request('promotion des ventes', '0', '15', "FR, Paris").json()

{'totalresults': None, 'results': None}